In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torchvision import models, transforms, datasets 
import torch.nn as nn
import torchattacks
import torch

In [5]:
# Define the transformation for images
transform = transforms.Compose([
    transforms.Resize((32, 32)),  # Resize to match the model input
    transforms.ToTensor()        # Convert to Tensor
])

# Load dataset
dataset = datasets.ImageFolder(root="data//tiny-imagenet-200/train", transform=transform)
loader = DataLoader(dataset, batch_size=30, shuffle=True)

#Get a batch of images and labels
images, labels = next(iter(loader))

In [6]:
#Get class labels
classes = dataset.classes
model = models.vgg16()

In [18]:
#Load a pre-trained VGG16 model.
model = models.vgg16(weights=models.VGG16_Weights.IMAGENET1K_V1)
num_classes = len(classes)

# Modify the classifier to suit the new task
model.classifier[5] = nn.Dropout(p=0.5)
model.classifier[6] = nn.Linear(in_features=4096, out_features=num_classes, bias=True)

#Update the classifier for the no. of classes in ur dataset
# model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
model.eval()
# attack = torchattacks.FGSM(model, eps=58/255)
attack = torchattacks.DeepFool(model, steps=50, overshoot=0.02)
adv_images = attack(images, labels)
diff = torch.norm(adv_images - images, p=2)
print("Difference between original and adversarial images (L2 norm):", diff.item())
adv_outputs = model(adv_images)
_, preds_perturbed = torch.max(adv_outputs, 1)
_, original_preds = torch.max(model(images), 1)



Difference between original and adversarial images (L2 norm): 0.0


In [10]:
for i in range(images.size(0)): 
    original_label = classes[original_preds[i].item()]
    adversarial_label = classes[preds_perturbed[i]]
    
    if original_label == adversarial_label:
        print(f"{original_label}: Label Not misclassified")
    else:
        print(f"{original_label}:{adversarial_label}=> Yes label is misclassified")

n04275548:n02927161=> Yes label is misclassified
n02129165:n02666196=> Yes label is misclassified
n09246464:n02666196=> Yes label is misclassified
n02226429:n04562935=> Yes label is misclassified
n07747607:n07711569=> Yes label is misclassified
n01629819:n02927161=> Yes label is misclassified
n02769748:n07711569=> Yes label is misclassified
n02769748:n02927161=> Yes label is misclassified
n04179913:n03937543=> Yes label is misclassified
n01855672:n07875152=> Yes label is misclassified
n03838899:n02699494=> Yes label is misclassified
n04149813:n04562935=> Yes label is misclassified
n02666196:n09256479=> Yes label is misclassified
n02917067:n09256479=> Yes label is misclassified
n04562935:n03014705=> Yes label is misclassified
n02814860:n02481823=> Yes label is misclassified
n03355925:n02124075=> Yes label is misclassified
n03400231:n09256479=> Yes label is misclassified
n03085013:n09256479=> Yes label is misclassified
n04532670:n01629819=> Yes label is misclassified
n02843684:n02699494=